In [2]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
import math
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


# Setup

In [44]:
x_train = np.load("ndsc-loaded/x_train.npy")
x_test = np.load("ndsc-loaded/x_test.npy")
y_train = np.load("ndsc-loaded/y_train.npy")
features = np.load("ndsc-loaded/features.npy")
test_features = np.load("ndsc-loaded/test_features.npy")
word_index = np.load("ndsc-loaded/word_index.npy").item()
embedding_matrix = np.load("ndsc-loaded/embedding_matrix.npy")

In [45]:
print(y_train)

[ 2 15 18 ... 25 25  7]


In [70]:
print(np.shape(embedding_matrix[25]))

(300,)


In [ ]:
# train_df = pd.read_csv("../input/ndsc-beginner/train.csv")[:50000]
# train_df, val_df = train_test_split(train_df, test_size=0.1)

In [ ]:
# # embdedding setup
# # Source https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html
# embeddings_index = {}
# f = open('../input/quora-insincere-questions-classification/embeddings/glove.840B.300d/glove.840B.300d.txt')
# for line in tqdm(f):
#     values = line.split(" ")
#     word = values[0]
#     coefs = np.asarray(values[1:], dtype='float32')
#     embeddings_index[word] = coefs
# f.close()

# # print('Found %s word vectors.' % len(embeddings_index))

In [ ]:
# # Convert values to embeddings
# def text_to_array(text):
#     empyt_emb = np.zeros(300)
#     text = text[:-1].split()[:30]
#     embeds = [embeddings_index.get(x, empyt_emb) for x in text]
#     embeds+= [empyt_emb] * (30 - len(embeds))
#     return np.array(embeds)

# # train_vects = [text_to_array(X_text) for X_text in tqdm(train_df["title"])]
# val_vects = np.array([text_to_array(X_text) for X_text in (val_df["title"][:4000])])
# val_y = np.array(val_df["Category"][:4000])

In [53]:
indices = np.arange(len(x_train)-1)

x_train[:2]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
         160,    5,  135,  126],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,  681,  501,   66,  765, 1046,
         158,  226,  210,  213]], dtype=int32)

In [66]:
indices = np.arange(len(x_train)-1)
np.random.shuffle(indices)
i = 0
x_train_batch_tokens = np.take(x_train, indices[i*batch_size:(i+1)*batch_size], axis = 0) 
x_train_batch = [[embedding_matrix[token] for token in title] for title in x_train_batch_tokens]
y_train_batch = np.take(y_train, indices[i*batch_size:(i+1)*batch_size])
print(indices)
print(x_train_batch_tokens)
print(np.shape(x_train_batch[:1]))
print(y_train_batch)

[124352 609812 515468 ... 583782 486744 101413]
[[    0     0     0 ...    26    42   304]
 [    0     0     0 ...   176 12665  3459]
 [    0     0     0 ...   210   213   931]
 ...
 [    0     0     0 ...    24   799  1155]
 [    0     0     0 ...  1469   165    93]
 [    0     0     0 ...   375   402   594]]
(1, 70, 300)
[35 31 12 26 43 25 31 42  4  3 22 18 31  0  3 12  3  3 43  4  3 32 18 25
 35 18  4 12 25 12 27 41  0  5  5 11 26 18 18 26  5 23 20  7 17  3  5 31
 31 13  4 18 34  2  3  3 18 31 28  8 35 27 27 31  4 26  4 18  3 31 31 35
 20 35  3 20 12  4  3  8  3 12  7  3  7 43 21  5 31 27 31  4  3 41 21  3
 33 25 35 26 32 27 35  3 41  6  9  8  5 18 20 21 41  2 28  7 18 25  5  7
  1 31 22 31  4 25  5 20]


In [67]:
# Data providers
batch_size = 128

indices = np.arange(len(x_train)-1)
# print(len(x_train))
# print(np.random.shuffle())

def batch_gen():
    n_batches = math.ceil(len(x_train) / batch_size)
    while True: 
        np.random.shuffle(indices)  # Shuffle the data.
#         print(indices)
        for i in range(n_batches):
#             texts = train_df.iloc[i*batch_size:(i+1)*batch_size, 1]
#             text_arr = np.array([text_to_array(text) for text in texts])
            
            x_train_batch_tokens = np.take(x_train, indices[i*batch_size:(i+1)*batch_size], axis = 0) 
            x_train_batch = [[embedding_matrix[token] for token in title] for title in x_train_batch_tokens]
            y_train_batch = np.take(y_train, indices[i*batch_size:(i+1)*batch_size])

            yield x_train_batch, y_train_batch

# Training

In [17]:
from keras.models import Sequential
from keras.layers import CuDNNLSTM, Dense, Bidirectional, Activation

In [25]:
model = Sequential()
model.add(Bidirectional(CuDNNLSTM(64, return_sequences=True),
                        input_shape=(30, 300)))
model.add(Bidirectional(CuDNNLSTM(64)))
model.add(Dense(58))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [68]:
mg = batch_gen()
model.fit_generator(mg, epochs=1,
                    steps_per_epoch=1000,
                    verbose=True)

Epoch 1/1
[272711 209676 512525 ... 303719 144284 586224]


AttributeError: 'list' object has no attribute 'shape'

# Inference

In [ ]:
# prediction part
batch_size = 256
def batch_gen(test_df):
    n_batches = math.ceil(len(test_df) / batch_size)
    for i in range(n_batches):
        texts = test_df.iloc[i*batch_size:(i+1)*batch_size, 1]
        text_arr = np.array([text_to_array(text) for text in texts])
        yield text_arr

test_df = pd.read_csv("../input/ndsc-beginner/test.csv")

all_preds = []
for x in tqdm(batch_gen(test_df)):
    all_preds.extend(model.predict(x).flatten())

In [ ]:
print(np.shape(all_preds)
y_te = [np.argmax(pred) for pred in all_preds]

submit_df = pd.DataFrame({"itemid": test_df["itemid"], "Category": y_te})
submit_df.to_csv("submission.csv", index=False)